In [3]:
from pprint import pprint
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
import wikipedia


Seleção de candidatos

In [23]:
try:
	searched_categories = wikipedia.page('python').categories
except wikipedia.DisambiguationError as e:
	for i, search_option in enumerate(e.options):
		print(f"{i}. {search_option}")

	choice_number = int(input("Escolha a busca e insira o número correspondente: "))
	chosen_option = e.options[choice_number]
	print(f"\nOpção escolhida: {chosen_option}\n")
	searched_categories = wikipedia.page(chosen_option).categories
	print(f"Categorias encontradas: {searched_categories}")

0. Pythonidae
1. Python (genus)
2. Python (mythology)
3. Python (programming language)
4. CMU Common Lisp
5. PERQ 3
6. Python of Aenus
7. Python (painter)
8. Python of Byzantium
9. Python of Catana
10. Python Anghelo
11. Python (Efteling)
12. Python (Busch Gardens Tampa Bay)
13. Python (Coney Island, Cincinnati, Ohio)
14. Python (automobile maker)
15. Python (Ford prototype)
16. Python (missile)
17. Python (nuclear primary)
18. Colt Python
19. PYTHON
20. Python (film)
21. Monty Python
22. Python (Monty) Pictures
23. Timon of Phlius
24. Cython
25. Pyton
26. Pithon

Opção escolhida: Python (programming language)

Categorias encontradas: ['All articles containing potentially dated statements', 'All articles with vague or ambiguous time', 'Articles containing potentially dated statements from June 2022', 'Articles containing potentially dated statements from October 2021', 'Articles with BNF identifiers', 'Articles with FAST identifiers', 'Articles with GND identifiers', 'Articles with J9U

Formando grupos de categorias que compartilham mesmo sufixo ou prefixo

In [24]:
#divide as strings das categorias
#analisa sufixos e prefixos de cada categoria
#cria dicionario que usar prefixos e sufixos como chave, e categorias como valor

def create_candidate_category_sets(categories):

	candidate_sets = dict()

	for category in categories:
		category_words = category.split()
		prefix = category_words[0].lower()
		sufix = category_words[-1].lower()
		if prefix not in candidate_sets:
			candidate_sets[prefix] = [category]
		else:
			candidate_sets[prefix].append(category)
		
		if prefix == sufix:
			continue

		if sufix not in candidate_sets:
			candidate_sets[sufix] = [category]
		else:
			candidate_sets[sufix].append(category)

	return candidate_sets


candidates = create_candidate_category_sets(searched_categories)
pprint(candidates)


{'1991': ['Programming languages created in 1991'],
 '2021': ['Articles containing potentially dated statements from October 2021',
          'Use dmy dates from November 2021'],
 '2022': ['Articles containing potentially dated statements from June 2022',
          'Vague or ambiguous time from May 2022'],
 'all': ['All articles containing potentially dated statements',
         'All articles with vague or ambiguous time'],
 'articles': ['Articles containing potentially dated statements from June 2022',
              'Articles containing potentially dated statements from October '
              '2021',
              'Articles with BNF identifiers',
              'Articles with FAST identifiers',
              'Articles with GND identifiers',
              'Articles with J9U identifiers',
              'Articles with LCCN identifiers',
              'Articles with NKC identifiers',
              'Articles with SUDOC identifiers',
              'Articles with example Python (programming 

Execução de queries SPARQL

In [25]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setQuery("""
	PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
	PREFIX foaf: <http://xmlns.com/foaf/0.1/>
	
	SELECT DISTINCT ?s ?label
	WHERE{
		?s a dbo:Film .
		?s dbp:name ?name .
		?s rdfs:label ?label .
		FILTER regex(?name, "^La La Land")
	}
	LIMIT 100
""")

sparql.setReturnFormat(JSON)
qresults = sparql.query().convert()
pprint(qresults)
for result in qresults["results"]["bindings"]:
	print(result["label"]["value"])
	print(result["s"]["value"])
	print("\n")


{'head': {'link': [], 'vars': ['s', 'label']},
 'results': {'bindings': [{'label': {'type': 'literal',
                                     'value': 'La La Land',
                                     'xml:lang': 'en'},
                           's': {'type': 'uri',
                                 'value': 'http://dbpedia.org/resource/La_La_Land'}},
                          {'label': {'type': 'literal',
                                     'value': 'لا لا لاند',
                                     'xml:lang': 'ar'},
                           's': {'type': 'uri',
                                 'value': 'http://dbpedia.org/resource/La_La_Land'}},
                          {'label': {'type': 'literal',
                                     'value': 'La La Land (film)',
                                     'xml:lang': 'cs'},
                           's': {'type': 'uri',
                                 'value': 'http://dbpedia.org/resource/La_La_Land'}},
                          {'